# Part 1

Pick three of your favorite books on one of your favorite subjects. At least one of the books should have more than one author. For each book, include the title, authors, and two or three other attributes that you find interesting. Take the information that you’ve selected about these three books, and separately create two files, one of which stores the books’ information in HTML (using an html table) and the other of which stores the books’ information in JSON format (e.g. “books.html” and “books.json”). Post the two source files to your GitHub repository.  Write Python code, using your packages of choice,
to load the information from each of the two files you’ve created into separate PANDAS data frames. Are the two data frames identical?

### JSON Solution

For JSON, I did it the first way, then saw that it is pretty ugly, so I switched to the second way.

In [ ]:
import pandas as pd

df_json = pd.read_json("https://raw.githubusercontent.com/MatthewFried/MAT5001/main/M9/books.json")
df_json.head()

,books
0,"{'id': '01', 'name': 'Bava Metzia', 'edition':..."
1,"{'id': '02', 'name': 'Bava Kama', 'edition': '..."
2,"{'id': '03', 'name': 'Bava Basra', 'edition': ..."


In [ ]:
from urllib.request import Request, urlopen
import json

import pandas as pd    

request=Request('https://raw.githubusercontent.com/MatthewFried/MAT5001/main/M9/books.json')
response = urlopen(request)
elevations = response.read()
data = json.loads(elevations)
df_json_pretty = pd.json_normalize(data['books'])
print(df_json_pretty)

   id         name  edition              author category
0  01  Bava Metzia  unknown  [Ravina, Rav Ashi]  Nezikin
1  02    Bava Kama  unknown  [Ravina, Rav Ashi]  Nezikin
2  03   Bava Basra  unknown  [Ravina, Rav Ashi]  Nezikin


### HTML Solution

The read_html function creates a list of dataframes. In this case, since I would like to use the data, I convert it to a regular dataframe.

In [ ]:
df_html = pd.read_html("https://raw.githubusercontent.com/MatthewFried/MAT5001/main/M9/books1.html")
df_html = pd.concat(df_html)
df_html.head()

,id,name,edition,author,category
0,1,Bava Metzia,unknown,Ravina and Rav Ashi,Nezikin
1,1,Bava Kama,unknown,Ravina and Rav Ashi,Nezikin
2,1,Bava Basra,unknown,Ravina and Rav Ashi,Nezikin


# Part 2



* Download the web page containing the Katz School’s AI Faculty
[information](https://www.yu.edu/katz/programs/graduate/artificial-intelligence/faculty).

* Create a BeautifulSoup class to parse the page you have downloaded.

* Within the downloaded content of the web page locate the div with class=”body”, and assign the results to a variable named faculty.

* Create a Pandas dataframe named faculty_info having columns name, title, bio, and teaching background. Each column should be capable of storing character strings.

* Within the HTML content stored within your faculty variable, locate and extract each faculty member’s name, title, bio, and teaching background and save these items to your faculty_info dataframe. You should have one dataframe row for each faculty member listed on the Katz School’s AI Faculty web page. 

### Solution

I download the data using BeautifulSoup and use a findall to assign the required data to nameList.  I note taht get_text() will get rid of all the html brackets.  I then convert the data to a list by parsing at the \n.  Then I add the data to a named table and then put in the dataframe.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

#download data and parse with BeautifulSoup
html = urlopen('https://www.yu.edu/katz/programs/graduate/artificial-intelligence/faculty')
bs = BeautifulSoup(html.read(), 'html.parser')

nameList = bs.find_all('div', {'class':'body'})

In [ ]:
#pandas dataframe

column_names = ['name','title','bio','teaching background']
df = pd.DataFrame(columns=column_names)
print(df)

Empty DataFrame
Columns: [name, title, bio, teaching background]
Index: []


In [ ]:
#convert nameList into clean text using get_text() from BeautifulSoup
#store values in a list

test = []
for name in nameList:
    test.append(name.get_text())

In [ ]:
#since it is stored as a 1 dimensional list that is a string, put it in a string
my_string = test[0]

#split the data on the \n 
test2 = [x.strip() for x in my_string.split('\n')]

In [ ]:
#check what we have

counter = 0
for k in test2:
  print(counter, k)
  counter += 1

0 Paul Russo, Ph.D. Vice Provost and Dean
1 Recent research examines computational methods in social networks and mobile applications by developing and empirically evaluating causal models. Studies focus on social sharing in online communities that are influenced by user motivations, trust, and network properties. This computational social science on the web promises to improve AI algorithms on platforms like Facebook, Instagram, Google Maps, and Tinder. In earlier projects, such as those at Texas Instruments, used Pattern Analysis and Machine Intelligence methods—e.g., clustering and Horn&Schunk algorithm—for image and streaming video analysis for machine vision and tracking moving objects. Other projects have bridged engineering and physics methods to build data collection systems that identify particles emitted as deep space materials degrade. Ethnographic studies in the use of technology in distributed science collaborations. Active peer reviewer. Co-PI for a $15M award to create t

In [ ]:
#create tables, assign accordingly, and then put in the dataframe
#we split at the comma so that we can get name and title from the same line

name = []
title = []
bio = []
teaching_background = []

counter = 0
for k in range(0,len(test2)-1,3):
  test3 = [x.strip() for x in test2[k].split(',')]
  name.append(test3[0])
  title.append(test3[1])
  bio.append(test2[k+1])
  teaching_background.append(test2[k+2])

df['name'] = name
df['title'] = title
df['bio'] = bio
df['teaching background'] = teaching_background

In [ ]:
print(df.head())

              name  ...                                teaching background
0       Paul Russo  ...  Teaches graduate courses in Social Computing, ...
1      Andy Catlin  ...  Teaches graduate courses in Analytics Programm...
2       Wonjun Lee  ...  Teaches graduate courses in Security Analytics...
3  Sergey Fogelson  ...  Teaches graduate courses in Computational Stat...
4  Lawrence Fulton  ...  Teaches graduate courses in Data Analytics, Ma...

[5 rows x 4 columns]


# Part 3

Our job is to use data from the nytimes API, download, and analyze it.  We first register for the nytimes API and then use our key to download data.  The data is stored in JSON format.  We use typical JSON python methods such as requests, get, load, and dump.  We leave the second code block below in case we want to see the JSON data.

In order to clean it better we use the third code block below and work through the data as commented.

In [2]:
#potential request, showing that the api works with a response of 200

import requests

response = requests.get("https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json?api-key=AcIwtjAMmQbO2Zd0QNcAJ8pGtZGZAbqB&offset=40")
print(response)

<Response [200]>


In [3]:
#make it readable
#sourced from https://www.dataquest.io/blog/python-api-tutorial/

import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

#jprint(response.json())

In [15]:
#redo request in order to make it more readable using a format that normalizes it in order to place in a dataframe

from urllib.request import Request, urlopen
import json

import pandas as pd    

rqst=Request("https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json?api-key=AcIwtjAMmQbO2Zd0QNcAJ8pGtZGZAbqB&offset=60")
rsp = urlopen(rqst)
elevations = rsp.read()
data = json.loads(elevations)

print(df_books.shape)
df_books = pd.json_normalize(data['results'])

(20, 3)


In [16]:
print(df_books.head())

                             title  ...                                            reviews
0                 12 BEAST, VOL. 3  ...  [{'book_review_link': '', 'first_chapter_link'...
1  12 SHORT HIKES NEAR LAKE PLACID  ...  [{'book_review_link': '', 'first_chapter_link'...
2                 12-ALARM COWBOYS  ...  [{'book_review_link': '', 'first_chapter_link'...
3                            12.21  ...  [{'book_review_link': 'https://www.nytimes.com...
4         1225 CHRISTMAS TREE LANE  ...  [{'book_review_link': '', 'first_chapter_link'...

[5 rows x 11 columns]


In [17]:
#we check our column names

print(df_books.columns)

Index(['title', 'description', 'contributor', 'author', 'contributor_note',
       'price', 'age_group', 'publisher', 'isbns', 'ranks_history', 'reviews'],
      dtype='object')


In [20]:
#we are doing a cursory analysis, and therefore drop anything we don't need

df_books.drop(['publisher','description','contributor','contributor_note','age_group','ranks_history', 'reviews','isbns'],axis = 1,inplace=True)
print(df_books.head())

                             title                 author price
0                 12 BEAST, VOL. 3                OKAYADO  0.00
1  12 SHORT HIKES NEAR LAKE PLACID             Phil Brown  0.00
2                 12-ALARM COWBOYS  Cora Seton and others  0.00
3                            12.21        Dustin Thomason  0.00
4         1225 CHRISTMAS TREE LANE        Debbie Macomber  0.00


In [21]:
#we double check we only have the columns we chose

df_books.columns

Index(['title', 'author', 'price'], dtype='object')

In [22]:
#below are the statistics for our data

df_books.describe()

,title,author,price
count,20,20,20
unique,18,16,4
top,1225 CHRISTMAS TREE LANE,James Patterson and Maxine Paetro,0.00
freq,2,2,16


In [23]:
#we output the average price and the number of zeros that are in price

price = []

for k in df_books['price']:
  price.append(float(k))

#the average price is below
import statistics 
print("Average price: ", statistics.mean(price))

#the number of zeros
print("Number of zeros: ", price.count(0))


Average price:  2.996
Number of zeros:  16


I see that this data is quite useless.  There are many errors (such as missing data and zeros for prices).  Even though I changed the offset to give more books, for some reason it is not.  As such, our minimal analysis has come to an end even without any charts, since this data is essentially useless. 